In [1]:
import json
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
from torch_geometric.utils import to_dense_batch
import torch.optim as optim

sys.path.append(os.path.abspath(".."))
from functions.utils_L2D import load_and_restore_parquet

#### 1. Download Data

- To download only one episode, use data_downloader(ep_num,...)

- To download a range of episodes, use data_downloader(min_ep,max_ep,...)

- Additionally, you can chnge the following directories:

    - tabular_data_dir = '../data/raw/L2D/tabular'

    - frames_dir = '../data/raw/L2D/frames'

In [2]:
from functions.load_data_L2D import data_downloader

data_downloader(32,n_secs=3,
                features={"tabular": True,
                          "frames": {
                                'observation.images.front_left': True,
                                'observation.images.left_backward': False,
                                'observation.images.left_forward': False,
                                'observation.images.map': False,
                                'observation.images.rear': False,
                                'observation.images.right_backward': False,
                                'observation.images.right_forward': False,
                            }
                        })

#### 2. Process Tabular Data & Add Tags

- To process only one episode, use process_tabular_data(ep_num,...)

- To process a range of episodes, use process_tabular_data(min_ep,max_ep,...)

- Additionally, you can change the following directories:

    - process_tabular_data:

        - source_dir = '../data/raw/L2D/tabular'

        - output_dir_processed = '../data/processed_data/L2D'

        - output_dir_tags = '../data/semantic_tags/L2D'

    - add_data_tags:

        - data_dir = '../data/processed/L2D'

        - tags_dir='../data/semantic_tags/L2D'

In [3]:
from functions.process_tabular_data_L2D import process_tabular_data
from functions.process_tags_L2D import add_data_tags

process_tabular_data(32,
                    overwrite=False, process_columns=True, 
                    process_osm=True, process_turning=False)
add_data_tags(32)

100%|██████████| 1/1 [00:00<00:00, 44.11it/s]


#### 3. Process Frames

- Set up with depth model with pip install git+https://github.com/apple/ml-depth-pro.git

- To process only one episode, use process_frames(ep_num,...)

- To process a range of episodes, use process_tabular_data(min_ep,max_ep,...)

- In this case, I have detection and depth set to False to make it run faster. If you have never run this code before, they should be set to true.

- Additionally, you can change the following directories:

    - input_base_dir = '../data/raw/L2D/frames',
    
    - output_base_dir = '../data/processed_frames/L2D'

- You can also include the following cameras:

    - "observation.images.front_left",
    - "observation.images.left_forward", 
    - "observation.images.right_forward",
    - "observation.images.right_backward",
    - "observation.images.rear",
    - "observation.images.left_backward"

**Note**: Including additional cameras will help with the depth, speed, etc. but CONSIDERABLY increases running time.

In [ ]:
from functions.process_frames_L2D import process_frames

process_frames(32,
               cameras_on=["observation.images.front_left"],
               run_dict={"detection": False,
                         "depth": False,
                         "speed": True},
                verbose=False)

For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.


🔍 Initializing Depth Pro...
✅ Depth Pro model already exists
✅ Depth Pro loaded on cpu
🤖 Initializing RF-DETR model...
Loading pretrain weights
✅ RF-DETR model loaded successfully

🔹 Processing episode 32 with config: {'detection': False, 'depth': False, 'speed': True}
  📷 Camera front_left: 5 frames found
  ✅ Processed 5 frames for front_left


{32: {'front_left': [{'frame': 'frame_00000.jpg',
    'detections': 0,
    'path': '../data/processed_frames/L2D/Episode000032/front_left_Segmented/frame_00000.jpg',
    'json': '../data/processed_frames/L2D/Episode000032/front_left_Annotations/frame_00000.json',
    'has_depth': False},
   {'frame': 'frame_00001.jpg',
    'detections': 0,
    'path': '../data/processed_frames/L2D/Episode000032/front_left_Segmented/frame_00001.jpg',
    'json': '../data/processed_frames/L2D/Episode000032/front_left_Annotations/frame_00001.json',
    'has_depth': False},
   {'frame': 'frame_00002.jpg',
    'detections': 0,
    'path': '../data/processed_frames/L2D/Episode000032/front_left_Segmented/frame_00002.jpg',
    'json': '../data/processed_frames/L2D/Episode000032/front_left_Annotations/frame_00002.json',
    'has_depth': False},
   {'frame': 'frame_00003.jpg',
    'detections': 0,
    'path': '../data/processed_frames/L2D/Episode000032/front_left_Segmented/frame_00003.jpg',
    'json': '../data/

#### 4. Generate Graphs

- 

In [5]:
from functions.graphs_L2D import combined_graph_viewer

ep_num = 32
with open(f"../data/graphical/L2D/{ep_num}_graph.json", "r") as f:
    graph_data = json.load(f)

cyto = combined_graph_viewer({'graph':graph_data})
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [6]:
#pip freeze > requirements.txt